In [1]:
from hermes.app import HermesApp

hermes_app = HermesApp()

In [3]:
hermes_app.get_csv_files(truncate=False)

[PosixPath('/data1/workspaces/phitha1/Hermes/data/shipment_questions_500.csv'),
 PosixPath('/data1/workspaces/phitha1/Hermes/data/shipments.csv'),
 PosixPath('/data1/workspaces/phitha1/Hermes/data/shipments_1000.csv')]

In [2]:
hermes_app.load_data(None, None, selected_file="data/shipments_1000.csv")

(       id    route warehouse  delivery_time  delay_minutes      delay_reason  \
 0       1  Route C       WH2           4.88             51     Customs Delay   
 1       2  Route A       WH4           4.77             45       Driver Rest   
 2       3  Route C       WH4           5.31            113           Traffic   
 3       4  Route B       WH4           5.83             32   Minor Breakdown   
 4       5  Route C       WH4           6.13            120           Traffic   
 ..    ...      ...       ...            ...            ...               ...   
 995   996  Route B       WH2           5.60            118           Weather   
 996   997  Route E       WH4           5.00             83       Driver Rest   
 997   998  Route B       WH4           5.45             52       Driver Rest   
 998   999  Route E       WH4           5.61            117           Traffic   
 999  1000  Route B       WH3           5.52            105  Mechanical Issue   
 
           date  on_time  

In [3]:
response = hermes_app._handle_general_chat("Show me the distribution of shipment delays.")

In [17]:
response.metadata["raw"]

DataFrameResponse(type='dataframe', value=       delay_reason  count
0              None    170
1           Traffic    157
2       Driver Rest    145
3  Mechanical Issue    143
4           Weather    137
5   Minor Breakdown    133
6     Customs Delay    115)

In [21]:
from pandasai.core.response import DataFrameResponse, NumberResponse, ChartResponse, StringResponse

In [23]:
response.__dict__

{'text': '🤖        delay_reason  count\n0              None    170\n1           Traffic    157\n2       Driver Rest    145\n3  Mechanical Issue    143\n4           Weather    137\n5   Minor Breakdown    133\n6     Customs Delay    115',
 'chart': None,
 'intent': 'general',
 'metadata': {'data_type': 'text',
  'raw': DataFrameResponse(type='dataframe', value=       delay_reason  count
  0              None    170
  1           Traffic    157
  2       Driver Rest    145
  3  Mechanical Issue    143
  4           Weather    137
  5   Minor Breakdown    133
  6     Customs Delay    115)},
 'timestamp': datetime.datetime(2025, 11, 6, 8, 53, 7, 296382),
 'success': True,
 'error': None,
 'data_type': 'text',
 'raw_result': DataFrameResponse(type='dataframe', value=       delay_reason  count
 0              None    170
 1           Traffic    157
 2       Driver Rest    145
 3  Mechanical Issue    143
 4           Weather    137
 5   Minor Breakdown    133
 6     Customs Delay    115)}

In [ ]:
from hermes.models import BaseResponse

response.chart.path if isinstance(response, BaseResponse) and response.chart else response.get("chart")
if isinstance(response, BaseResponse):
    if response.chart:
        return response.chart.path
else:
        return response.get("chart")

AttributeError: 'GeneralResponse' object has no attribute 'get'

In [29]:
response = hermes_app._handle_prediction_chat("Predict the delay rate for next week.")

TypeError: argument of type 'pydantic_core._pydantic_core.ValidationInfo' is not iterable

In [34]:
from hermes.config import CHARTS_DIR, DATA_DIR, QUESTIONS_FILE, SHIPMENTS_FILE, llm
from hermes.visualizer import HermesVisualizer
visualizer = HermesVisualizer(charts_dir="/data1/workspaces/phitha1/Hermes/exports/charts")

In [36]:
visualizer.get_latest_chart()

'/data1/workspaces/phitha1/Hermes/exports/charts/temp_chart_a22db8c9-d29c-4c67-a3e0-c664fbeacf8c.png'